In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
df_loinc = pd.read_csv('data/loinc.csv')

/Users/ericwu/Applications/miniconda3/envs/gensim3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (8,15,17,18,25,26,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_loinc.columns

Index(['LOINC_NUM', 'COMPONENT', 'PROPERTY', 'TIME_ASPCT', 'SYSTEM',
       'SCALE_TYP', 'METHOD_TYP', 'CLASS', 'VersionLastChanged', 'CHNG_TYPE',
       'DefinitionDescription', 'STATUS', 'CONSUMER_NAME', 'CLASSTYPE',
       'FORMULA', 'SPECIES', 'EXMPL_ANSWERS', 'SURVEY_QUEST_TEXT',
       'SURVEY_QUEST_SRC', 'UNITSREQUIRED', 'SUBMITTED_UNITS', 'RELATEDNAMES2',
       'SHORTNAME', 'ORDER_OBS', 'CDISC_COMMON_TESTS', 'HL7_FIELD_SUBFIELD_ID',
       'EXTERNAL_COPYRIGHT_NOTICE', 'EXAMPLE_UNITS', 'LONG_COMMON_NAME',
       'UnitsAndRange', 'DOCUMENT_SECTION', 'EXAMPLE_UCUM_UNITS',
       'EXAMPLE_SI_UCUM_UNITS', 'STATUS_REASON', 'STATUS_TEXT',
       'CHANGE_REASON_PUBLIC', 'COMMON_TEST_RANK', 'COMMON_ORDER_RANK',
       'COMMON_SI_TEST_RANK', 'HL7_ATTACHMENT_STRUCTURE',
       'EXTERNAL_COPYRIGHT_LINK', 'PanelType', 'AskAtOrderEntry',
       'AssociatedObservations', 'VersionFirstReleased',
       'ValidHL7AttachmentRequest'],
      dtype='object')

In [4]:
cols = ['LOINC_NUM', 'COMPONENT', 'PROPERTY', 'TIME_ASPCT', 'SYSTEM', 'SCALE_TYP', 'METHOD_TYP', 'CLASS',
       'SHORTNAME', 'LONG_COMMON_NAME', 'RELATEDNAMES2']

In [5]:
df_loinc[cols].sample(5)

,LOINC_NUM,COMPONENT,PROPERTY,TIME_ASPCT,SYSTEM,SCALE_TYP,METHOD_TYP,CLASS,SHORTNAME,LONG_COMMON_NAME,RELATEDNAMES2
3764,1331-8,V Ab,PrThr,Pt,Ser/Plas,Ord,NaN,BLDBK,V Ab SerPl Ql,V Ab [Presence] in Serum or Plasma,ABS; Aby; Antby; Anti; Antibodies; Antibody; A...
30070,38268-9,Bone density,Find,Pt,Whole body>Skeletal system,Doc,DXA,RAD,Skeletal DXA BMD,Skeletal system DXA Bone density,BMD; bod; Bodies; Bone Densiometry; Bone Densi...
21105,29328-2,Xylose^2.5H post dose xylose PO,MCnc,Pt,Bld,Qn,NaN,CHAL,Xylose 2.5h p Xyl PO Bld-mCnc,Xylose [Mass/volume] in Blood --2.5 hours post...,2.5h p Xyl PO; After; Blood; CHEMISTRY.CHALLEN...
46552,52403-3,Activities permitted,Find,Pt,Speech therapy treatment plan,Nar,NaN,ATTACH.REHAB.SPEECH,NaN,"Speech therapy treatment plan, Activities perm...",ATTACH; ATTACH.REHAB; ATTACH.REHAB.SPEECH; Fin...
77288,80663-8,4-Hydroxy 3-Nitrophenylacetate,MCnc,Pt,Urine,Qn,NaN,DRUG/TOX,HNPAA Ur-mCnc,4-Hydroxy 3-Nitrophenylacetate (HNPAA) [Mass/v...,DRUG/TOXICOLOGY; Drugs; HNPAA; Hydrox; III; Le...


### Columns for embeddings

The goal is collect short sentences that can be used to generate the embeddings.

The following columns contain values that can be used directly as sentences for generating embeddings: 

    ['COMPONENT', 'SHORTNAME', 'LONG_COMMON_NAME', 'SYSTEM', 'CLASS']

The column `RELATEDNAMES` must split by ";" into multiple sentences.

The final out put will be a data frame of 3 columns: `['LOINC_NUM', 'SENTENCE', 'ORG_COLUMN']`

* LOINC_NUM -- The LOINC number is the label
* SENTENCE -- The sentences that will be used to generate the embeddings
* ORG_COLUMN -- The original column containing the sentence

In [6]:
df_component = df_loinc[['LOINC_NUM', 'COMPONENT']]
df_short_name = df_loinc[['LOINC_NUM', 'SHORTNAME']]
df_long_name = df_loinc[['LOINC_NUM', 'LONG_COMMON_NAME']]
df_system = df_loinc[['LOINC_NUM', 'SYSTEM']]
df_class = df_loinc[['LOINC_NUM', 'CLASS']]
df_related = df_loinc[['LOINC_NUM', 'RELATEDNAMES2']]

In [7]:
# Split the related names by ';' and save them into separate rows
rows = []
for index, row in df_related.iterrows():
    loinc_num = row['LOINC_NUM']
    names = row['RELATEDNAMES2']
    for name in names.split(';'):
        name = name.strip()
        new_row = {
            'LOINC_NUM': loinc_num,
            'RELATED_NAME': name
        }
        rows.append(new_row)
df_related = DataFrame(rows)

In [8]:
df_related.shape

(1791428, 2)

In [9]:
df_related.head(5)

,LOINC_NUM,RELATED_NAME
0,10000-8,Cardiac
1,10000-8,Durat
2,10000-8,ECG
3,10000-8,EKG.MEASUREMENTS
4,10000-8,Electrocardiogram


In [10]:
rows[:30]

[{'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Cardiac'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Durat'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'ECG'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'EKG.MEASUREMENTS'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Electrocardiogram'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Electrocardiograph'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Hrt'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': "Painter's colic"},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'PB'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Plumbism'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Point in time'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'QNT'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Quan'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Quant'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'Quantitative'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'R prime'},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': "R' wave dur L-AVR"},
 {'LOINC_NUM': '10000-8', 'RELATED_NAME': 'R w

In [11]:
df_component.to_csv('data/loinc-component.csv', index=False)
df_short_name.to_csv('data/loinc-short-name.csv', index=False)
df_long_name.to_csv('data/loinc-long-name.csv', index=False)
df_system.to_csv('data/loinc-system.csv', index=False)
df_class.to_csv('data/loinc-class.csv', index=False)
df_related.to_csv('data/loinc-related-name.csv', index=False)